In [1]:
# -*- coding: utf-8 -*-
"""
@author: Etienne Kras, open in SDB_env
"""

# imports 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import gdal
import os
import sys
import ee
import geemap
import geojson
import datetime
import time

from geojson import Feature, FeatureCollection, dump
from shapely.geometry import Polygon
from matplotlib.colors import ListedColormap
from scipy import stats
from pathlib import Path
from eepackages.applications import bathymetry
from eepackages.utils import download_image_collection, download_image_collection_thumb
from dateutil.relativedelta import *
from google.oauth2.service_account import Credentials

In [ ]:
# project toggles
file_fol = r'p:\satellite-derived-bathymetry\data' # input folder for the output data
xtile = 'x264' # fill in when toggled spatial
ytile = 'y165' # fill in when toggled spatial 
draw_AoI = 0
project_name = 'Friese_Zeegat'

# composite image toggles
start_date = '2015-01-01' # start date of the composites
stop_date = '2020-10-01' # end date of the composites
compo_int = 3 # composite interval [months]
compo_len = 24 # composite length [months]
scale = 19.109  # output resolution of the image [m]

# Pre-processing using the API (TO BE FINISHED)

In [ ]:
# specify or load Area of Interest (AoI)

Map = geemap.Map(center=(52.643246, 5.060993), zoom=8) # initialize map with base in Hoorn

if draw_AoI == 1:
    print('Please draw a polygon somewhere in a water body') # identifier
if draw_AoI == 0:
    # open AoI
    print('Loading and visualizing AoI') #identifier
    AoIee = geemap.geojson_to_ee(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson'))
    Map.addLayer(AoIee, {}, 'AoI')

Map # show map

In [ ]:
# construct the AoI if polygon is drawn in cell above

if draw_AoI == 1:
    
    print('Constructing AoI from drawn polygon') # identifier
    
    # get AoI 
    AoIee = ee.FeatureCollection(Map.draw_features) # make featurecollection
    AoI = Polygon(AoIee.getInfo()['features'][0]['geometry']['coordinates'][0]) # create AoI shapefile

    # export AoI
    features = []
    features.append(Feature(geometry=AoI, properties={"AoI": project_name}))
    feature_collection = FeatureCollection(features)
    with open(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson'), 'w') as f: # geojson
        dump(feature_collection, f)
    gdr = gpd.GeoDataFrame({'properties':{'AoI': project_name}, 'geometry': AoI}, crs='EPSG:4326') #shp
    gdr.to_file(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.shp'))
    bounds = ee.Geometry.Polgyon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    
if draw_AoI == 0:
    print('Reconstructing AoI from loaded file')
    # get AoI
    with open(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson')) as f:
        AoIjson = geojson.load(f)
    try: # drawn polygon in this script
        AoI = Polygon(AoIjson['features'][0]['geometry']['coordinates'][0]) 
    except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
        AoI = Polygon(AoIjson['features'][0]['geometry']['coordinates'][0][0])
    bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])

In [ ]:
# create composite subtidal images 

# image timeframes
sdate = datetime.datetime.strptime(start_date,'%Y-%m-%d')
edate = datetime.datetime.strptime(stop_date,'%Y-%m-%d')
window_length = int((edate.year-sdate.year)*12+(edate.month-sdate.month))
srangedates = pd.date_range(start_date, freq='%sMS'%(compo_int), periods=int((window_length-compo_len)/3)+1).strftime('%Y-%m-%d').tolist()
erangedates = pd.date_range((sdate+relativedelta(months=compo_len)).strftime('%Y-%m-%d'), freq='%sMS'%(compo_int), periods=int((window_length-compo_len)/3)+1).strftime('%Y-%m-%d').tolist()

sdb = bathymetry.Bathymetry() # initialize sdb instance

# save composite ee.Images to a list (note, these are not yet processed)
image_list = []
for starts, ends in zip(srangedates, erangedates):
    
    image = sdb.compute_inverse_depth(
        bounds=bounds,
        start=starts,
        stop=ends,
        scale=scale,
        missions=['S2', 'L8'],
        filter_masked=True,
        skip_neighborhood_search=False,
    ).clip(bounds) # clip to bounds only
    
    image_list.append(image)

In [ ]:
# create empty imagecollection
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_id = f'users/{user_name}/RSW_SDB/%s'%(project_name)
try:
    ee.data.createAsset({'type': 'Folder'}, '/'.join(asset_id.split('/')[:-1]))
    #ee.data.createAsset({'type': 'ImageCollection'}, asset_id)
except:
    print('Asset was already created, cannot overwrite')

In [ ]:
# save composites various locations (depending on your preference, select )
store_asset = 1 # toggle 1 to save to assets (Google Earth Engine)
store_gcs = 0 # toggle 1 to save to cloud storage (Google Earth Engine)

# TODO: fix writing to an image in imagecollection, not only one image..

# to assets --> visualize in Code Editor UI
if store_asset == 1:
    task = ee.batch.Export.image.toAsset(**{
        'image': image_list[0],
        'description': '%s_%s_%s'%(project_name, srangedates[0], erangedates[0]),
        'scale': scale,
        'region': bounds,
        'assetId': asset_id,
        'maxPixels': 1e11,
        'crs': 'EPSG:3857'
    })
    task.start()

# to cloud storage --> visualize in QGIS / ArcGIS (download via cloud storage platform or enable local downloading)
# TODO: use the service account for this
if store_gcs == 1:
    #task = ee.batch.Export.image.toCloudStorage(**{
    #    'image': ee.Image(img), 
    #    'description': asset_name + '_' + ee.Image(img).get('system:index').getInfo(),
    #    'scale': ee.Image(img).projection().nominalScale().getInfo(),
    #    'region': ee.Image(img).geometry().getInfo()['coordinates'],
    #    'fileNamePrefix': project + '/' + asset_name + '_' + ee.Image(img).get('system:index').getInfo(),
    #    'fileFormat': 'GeoTIFF',
    #    'bucket': 'jip_calm_sdb',
    #    'formatOptions': {'cloudOptimized': True}, # enables easy QGIS plotting
    #    'maxPixels': 1e10,
    #    'crs': 'EPSG:3857',
    #})
    #task.start()

In [ ]:
local_store = 0 # toggle 1 to save to local drive

In [ ]:
task_status = task.status()
while task_status['state'] in ['READY', 'RUNNING']:
    print(task_status)
    time.sleep(30)
    task_status = task.status()
else:
    print(task.status())

In [ ]:
# show image (indicative)
AoIcent = AoIee.geometry().centroid().coordinates()
Map = geemap.Map(center=(AoIcent.get(1).getInfo(), AoIcent.get(0).getInfo()), zoom=12) 
Map.addLayer(AoIee, {}, 'AoI')
Map.addLayer(image.select('red'), {}, 'composite')#, { "min": min, "max": max }, 'red-green-blue')
Map 

In [ ]:
# also put in toggle for intertidal or subtidal assessment, for intertidal see example: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/intertidal_bathymetry.ipynb

In [ ]:
# for service account GCS writing see for example: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/test_service_user.ipynb

In [ ]:
# finish subtidal bathy downloading (see for example: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/bathymetry_subtidal.ipynb)

In [ ]:
# enable single image downloading in the created composites.. (see for example: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/Download_SDB_ImageCollection.ipynb)

In [ ]:
# see scheme at p:\satellite-derived-bathymetry\ppt\acces_api.pptx what this scripts needs to do